### 读取数据

In [2]:
import pandas as pd
import numpy as np
import scipy.io as sio
import scipy.sparse as ss

from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
import pickle

In [2]:
# 读取数据train数据集
dpath = "../Data/"
trainTemp = pd.read_csv(dpath+"train.csv", iterator=True)
# 设置分块参数
loop = True
chunkSize = 100000
chunks = []

while loop:
    try:
        chunk = trainTemp.get_chunk(chunkSize)
        chunks.append(chunk)
    except StopIteration:
        loop = False
        print("Iteration is stopped.")
train = pd.concat(chunks, ignore_index=True)
print(len(train))

Iteration is stopped.
7377418


In [3]:
# 读取songs数据集
songs = pd.read_csv(dpath+"songs.csv")
print(len(songs))

2296320


In [4]:
train_songs = pd.merge(train, songs, how="outer")
train_songs.head()

,msno,song_id,source_system_tab,source_screen_name,source_type,target,song_length,genre_ids,artist_name,composer,lyricist,language
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1.0,206471.0,359,Bastille,Dan Smith| Mark Crew,NaN,52.0
1,e5Ezre9HPuPos+CXQXtmo32E/hHIZTMmo6jG3yRf6UA=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,my library,Local playlist more,local-playlist,1.0,206471.0,359,Bastille,Dan Smith| Mark Crew,NaN,52.0
2,pouJqjNRmZOnRNzzMWWkamTKkIGHyvhl/jo4HgbncnM=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,discover,Online playlist more,online-playlist,0.0,206471.0,359,Bastille,Dan Smith| Mark Crew,NaN,52.0
3,sSexP400TJOZRhx3JB+0s9cqrCnqrlV51B9njoKR1II=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,discover,Online playlist more,online-playlist,0.0,206471.0,359,Bastille,Dan Smith| Mark Crew,NaN,52.0
4,hKdGiUKHVqKkXGHLrc+EzdSW6q0ERAJ2Cs7/L1N0Ae4=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,discover,NaN,online-playlist,0.0,206471.0,359,Bastille,Dan Smith| Mark Crew,NaN,52.0


In [5]:
train_songs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9313824 entries, 0 to 9313823
Data columns (total 12 columns):
msno                  object
song_id               object
source_system_tab     object
source_screen_name    object
source_type           object
target                float64
song_length           float64
genre_ids             object
artist_name           object
composer              object
lyricist              object
language              float64
dtypes: float64(3), object(9)
memory usage: 923.8+ MB


In [6]:
print(train_songs.isnull().sum())

msno                  1936406
song_id                     0
source_system_tab     1961255
source_screen_name    2351210
source_type           1957945
target                1936406
song_length               114
genre_ids              205338
artist_name               114
composer              2591558
lyricist              4855358
language                  150
dtype: int64


### 清理数目比较少的空值

In [7]:
filterData = ['song_length', 'artist_name', 'language', 'target']
TrainSongs = train_songs.dropna(subset = filterData)
TrainSongs = TrainSongs.drop(["composer", "lyricist"], axis=1)
print(TrainSongs.isnull().sum())
len(TrainSongs)

msno                       0
song_id                    0
source_system_tab      24849
source_screen_name    414794
source_type            21539
target                     0
song_length                0
genre_ids             118341
artist_name                0
language                   0
dtype: int64


7377268

### 将数目比较少的变量进行合并

In [8]:
TrainSongs["genre_ids"].value_counts()

465                            3717690
458                            1261208
921                             350311
1609                            315089
444                             243564
1259                            181713
2022                            152545
359                             109513
2122                             71859
139                              65827
451                              63818
437                              58530
958                              56125
786                              43309
1616|1609                        42306
465|1259                         35352
1011                             33496
444|1259                         32610
921|465                          29746
139|125|109                      28031
2157                             26430
726                              21807
921|458                          19760
465|458                          16925
947                              15516
691                      

In [9]:
splitGenre = TrainSongs.drop('genre_ids', axis=1).join(
    TrainSongs['genre_ids'].str.split('|', expand=True).stack().reset_index(
        level=1, drop=True).rename('genre_ids'))

In [1]:
splitGenre["genre_ids"].value_counts()[:50]

NameError: name 'splitGenre' is not defined

In [11]:
print(splitGenre.isnull().sum())
print(splitGenre["source_system_tab"].value_counts())
print(splitGenre["source_type"].value_counts())
TrainSongs["source_screen_name"].value_counts()

msno                       0
song_id                    0
source_system_tab      26103
source_screen_name    436065
source_type            22476
target                     0
song_length                0
artist_name                0
language                   0
genre_ids             118341
dtype: int64
my library      3860572
discover        2314052
search           656772
radio            508285
listen with      218239
explore          178500
notification       6498
settings           2273
Name: source_system_tab, dtype: int64
local-library             2361269
online-playlist           2091690
local-playlist            1134915
radio                      514876
album                      508718
top-hits-for-artist        445326
song                       259825
song-based-playlist        218168
listen-with                198193
topic-article-playlist      12012
artist                       3143
my-daily-playlist             683
Name: source_type, dtype: int64


Local playlist more     3228136
Online playlist more    1294679
Radio                    474449
Album more               420131
Search                   298478
Artist more              252425
Discover Feature         244244
Discover Chart           213658
Others profile more      201794
Discover Genre            82200
My library                75978
Explore                   72341
Unknown                   54170
Discover New              15955
Search Trends             13632
Search Home               13482
My library_Search          6451
Self profile more           212
Concert                      47
Payment                      12
Name: source_screen_name, dtype: int64

In [12]:
col_origin = splitGenre.columns
otherSouTab = ["notification", "settings", np.nan]
otherSouType = ["topic-article-playlist", "artist", "my-daily-playlist", np.nan]
otherSouScre = ["Payment", "Concert", "Self profile more", "My library_Search",
                "Search Home", "Discover New"]
otherLanguage = [ 24.0, 59.0, 45.0, 38.0]
cols = np.array(splitGenre).tolist()
for col in cols:
    if col[2] in otherSouTab:
        col[2] = "Other"
    if col[3] in otherSouScre:
        col[3] = "Other"
    if col[4] in otherSouType:
        col[4] = "Other"
    if col[8] in otherLanguage:
        col[8] = "Ohter"
TrainSongs = pd.DataFrame(columns = col_origin, data = cols)

In [13]:
print(TrainSongs["source_system_tab"].value_counts())
print(TrainSongs["source_type"].value_counts())
TrainSongs.isnull().sum()

my library     3860572
discover       2314052
search          656772
radio           508285
listen with     218239
explore         178500
Other            34874
Name: source_system_tab, dtype: int64
local-library          2361269
online-playlist        2091690
local-playlist         1134915
radio                   514876
album                   508718
top-hits-for-artist     445326
song                    259825
song-based-playlist     218168
listen-with             198193
Other                    38314
Name: source_type, dtype: int64


msno                       0
song_id                    0
source_system_tab          0
source_screen_name    436065
source_type                0
target                     0
song_length                0
artist_name                0
language                   0
genre_ids             118341
dtype: int64

In [14]:
feature_count = TrainSongs['genre_ids'].value_counts()  
l = len(feature_count.values)
small_list=[]  # 将样本数少于10000的变量放入list
for i in range(l):
    if feature_count.values[i] <= 10000:
        small_list.append(feature_count.index[i])

    
m = len(TrainSongs)
feature_values = TrainSongs["genre_ids"].values
for j in range(m):
    for i in small_list:
        if i==feature_values[j]:
            feature_values[j] = "Other"
TrainSongs["genre_ids"]=feature_values

In [15]:
print(TrainSongs["genre_ids"].value_counts())
print(TrainSongs["source_system_tab"].value_counts())
print(TrainSongs["source_type"].value_counts())
print(TrainSongs["source_screen_name"].value_counts())
print(TrainSongs["language"].value_counts())

465      3848078
458      1311333
921       410660
1609      374123
444       282518
1259      251838
2022      166537
Other     144066
359       114999
139       102186
2122       80357
786        67423
451        63858
958        60497
437        59310
1616       55285
1011       40038
947        34147
109        33270
125        28899
726        27202
2157       27128
691        16250
430        15792
1152       14309
829        12033
798        10817
Name: genre_ids, dtype: int64
my library     3860572
discover       2314052
search          656772
radio           508285
listen with     218239
explore         178500
Other            34874
Name: source_system_tab, dtype: int64
local-library          2361269
online-playlist        2091690
local-playlist         1134915
radio                   514876
album                   508718
top-hits-for-artist     445326
song                    259825
song-based-playlist     218168
listen-with             198193
Other                    38314
Na

### Onehot编码

In [16]:
"""
TrainSongs["source_system_tab"].astype("object")
TrainSongs["source_type"].astype("object")
TrainSongs["source_screen_name"].astype("object")
TrainSongs[ "genre_ids"].astype("object")
TrainSongs["language"].astype("object")
tabCat = pd.get_dummies(TrainSongs["source_system_tab"], prefix="source_system_tab")
typeCat = pd.get_dummies(TrainSongs["source_type"], prefix="source_type")
nameCat = pd.get_dummies(TrainSongs["source_screen_name"], prefix="source_screen_name")
geidsCat = pd. get_dummies(TrainSongs["genre_ids"], prefix="genre_ids")
langCat = pd.get_dummies(TrainSongs["language"], prefix="language")
one_TrainSongs = [TrainSongs, tabCat,typeCat, nameCat, geidsCat, langCat]
TrainSongs = pd.concat(one_TrainSongs, axis=1).drop(["source_system_tab", "source_type", "source_screen_name",
                                                     "genre_ids", "language"], axis=1)
TrainSongs.head()
"""

'\nTrainSongs["source_system_tab"].astype("object")\nTrainSongs["source_type"].astype("object")\nTrainSongs["source_screen_name"].astype("object")\nTrainSongs[ "genre_ids"].astype("object")\nTrainSongs["language"].astype("object")\ntabCat = pd.get_dummies(TrainSongs["source_system_tab"], prefix="source_system_tab")\ntypeCat = pd.get_dummies(TrainSongs["source_type"], prefix="source_type")\nnameCat = pd.get_dummies(TrainSongs["source_screen_name"], prefix="source_screen_name")\ngeidsCat = pd. get_dummies(TrainSongs["genre_ids"], prefix="genre_ids")\nlangCat = pd.get_dummies(TrainSongs["language"], prefix="language")\none_TrainSongs = [TrainSongs, tabCat,typeCat, nameCat, geidsCat, langCat]\nTrainSongs = pd.concat(one_TrainSongs, axis=1).drop(["source_system_tab", "source_type", "source_screen_name",\n                                                     "genre_ids", "language"], axis=1)\nTrainSongs.head()\n'

### 将Train_Songs保存起来

In [17]:
TrainSongs.to_csv(dpath +'TrainSongs.csv',index=False,header=True)